In [155]:
#importing some basic stuff for analysis
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
# linear algebra
import numpy as np 

In [156]:
#importing the data
titanic_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
### want to make sure I read the data 
#print(X)
#read it and it was correct! 

In [157]:
### dropping useless stuff 
#after reading it there was some useless things
#the name does not affect the survival 
# NaN columns for cabin were a lot, pretty useless too, etc

#having issues with the code below so trying to see what is going on with this line
#print("TITANIC DATA COLS")
#print(titanic_data.columns.values)
#print("TEST DATA COLS")
#print(test_data.columns.values)
#OMG! THEY DONT HAVE THE SAME COLS :/ so get the same cols here lady!! 

#titanic_data = titanic_data.drop('Survived', axis=1)
#I think I might need to keep passenger ID on the test set for submission lol
test_data = test_data.drop(['Embarked','Name','Cabin'], axis=1)

titanic_data = titanic_data.drop(['PassengerId','Embarked','Name','Cabin'], axis=1)
#making sure everything that was supposed to be dropped was dropped
print("Checking test data drop")
print(test_data.columns.values)
print("Checking titanic test data")
print(titanic_data.columns.values)

Checking test data drop
['PassengerId' 'Pclass' 'Sex' 'Age' 'SibSp' 'Parch' 'Ticket' 'Fare']
Checking titanic test data
['Survived' 'Pclass' 'Sex' 'Age' 'SibSp' 'Parch' 'Ticket' 'Fare']


In [158]:
#going to use a DTC, which in this case is part of a python package lib
model = DecisionTreeClassifier()

In [159]:
#okay now its time for some data training in this area lol, at last
data = [titanic_data, test_data]

#filling in the gap for the age stuff since not all vals are actually filled out 
for dataset in data:
    mean = titanic_data["Age"].mean()
    std = test_data["Age"].std()
    is_null = dataset["Age"].isnull().sum()
    # compute random numbers between the mean, std and is_null
    rand_age = np.random.randint(mean - std, mean + std, size = is_null)
    # fill NaN values in Age column with random values generated
    age_slice = dataset["Age"].copy()
    age_slice[np.isnan(age_slice)] = rand_age
    dataset["Age"] = age_slice
    dataset["Age"] = titanic_data["Age"].astype(int)
titanic_data["Age"].isnull().sum()

#need to recheck what is going on here lol because I have lost myself in all the data again 
#below is just to print the data info, but once needed I commented it out 
#titanic_data.info()

#wow a lil probablem is that for the comparison they are not all the same types 
#aka you have comparison between floats, ints and objects lol terrible! 
#let's start by changing fare into an int! 
data = [titanic_data, test_data]

#CHANGING FARE FLOAT--> INT 
for dataset in data:
    #astype will cast into a different guy 
    dataset['Fare'] = dataset['Fare'].fillna(0)
    dataset['Fare'] = dataset['Fare'].astype(int)
    
#CHANGING SEX CHAR-->BOOL, INT
genders = {"male": 0, "female": 1}
data = [titanic_data, test_data]

for dataset in data:
    dataset['Sex'] = dataset['Sex'].map(genders)
    
#it says ticket is an object
titanic_data['Ticket'].describe()
#this one has 681 unique things, not sure what that means but I can't think of a way to convert this into just one thing
titanic_data = titanic_data.drop(['Ticket'], axis=1)
test_data = test_data.drop(['Ticket'], axis=1)

In [160]:
#okay so now we create the cateogies 

#but before I break up into cats I want to see what I still have remaining of my data lol 
#again too much data I keep forgetting what I am doing 
print("Checking test data avail")
print(test_data.columns.values)
print("Checking titanic data avail")
print(titanic_data.columns.values)

data = [titanic_data, test_data]
for dataset in data:
    dataset['Age'] = dataset['Age'].astype(int)
    dataset.loc[ dataset['Age'] <= 11, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 11) & (dataset['Age'] <= 18), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 18) & (dataset['Age'] <= 22), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 22) & (dataset['Age'] <= 27), 'Age'] = 3
    dataset.loc[(dataset['Age'] > 27) & (dataset['Age'] <= 33), 'Age'] = 4
    dataset.loc[(dataset['Age'] > 33) & (dataset['Age'] <= 40), 'Age'] = 5
    dataset.loc[(dataset['Age'] > 40) & (dataset['Age'] <= 66), 'Age'] = 6
    dataset.loc[ dataset['Age'] > 66, 'Age'] = 6

# let's see how it's distributed 
titanic_data['Age'].value_counts()

#same thing for fare here again
#pretty sure I don't have to keep declaring this but whatevs 
data = [titanic_data, test_data]

#now taking care of categorizing the fare!
for dataset in data:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[(dataset['Fare'] > 31) & (dataset['Fare'] <= 99), 'Fare']   = 3
    dataset.loc[(dataset['Fare'] > 99) & (dataset['Fare'] <= 250), 'Fare']   = 4
    dataset.loc[ dataset['Fare'] > 250, 'Fare'] = 5
    dataset['Fare'] = dataset['Fare'].astype(int)

Checking test data avail
['PassengerId' 'Pclass' 'Sex' 'Age' 'SibSp' 'Parch' 'Fare']
Checking titanic data avail
['Survived' 'Pclass' 'Sex' 'Age' 'SibSp' 'Parch' 'Fare']


In [161]:
#actually doing some comparisons with the data now
X_train = titanic_data.drop("Survived", axis=1)
Y_train = titanic_data["Survived"]
X_test  = test_data.drop("PassengerId", axis=1).copy()
#fit all this crap into the DT 
decision_tree = DecisionTreeClassifier() 
decision_tree.fit(X_train, Y_train)  
Y_pred = decision_tree.predict(X_test)  
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)

In [162]:
#for kaggle submission
submission = pd.DataFrame({
        "PassengerId": test_data["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv('titanic.csv', index=False)
print('Exported')

Exported
